<a href="https://colab.research.google.com/github/ritwiks9635/New_Project_Ideas/blob/main/Speaker_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Speaker Recognition**

## Introduction

This example demonstrates how to create a model to classify speakers from the
frequency domain representation of speech recordings, obtained via Fast Fourier
Transform (FFT).

It shows the following:

- How to use `tf.data` to load, preprocess and feed audio streams into a model
- How to create a 1D convolutional network with residual
connections for audio classification.

Our process:

- We prepare a dataset of speech samples from different speakers, with the speaker as label.
- We add background noise to these samples to augment our data.
- We take the FFT of these samples.
- We train a 1D convnet to predict the correct speaker given a noisy FFT speech sample.

Note:

- This example should be run with TensorFlow 2.3 or higher, or `tf-nightly`.
- The noise samples in the dataset need to be resampled to a sampling rate of 16000 Hz
before using the code in this example. In order to do this, you will need to have
installed `ffmpg`.

#[Dataset](https://www.kaggle.com/datasets/kongaevans/speaker-recognition-dataset)

In [ ]:
!unzip /content/https:/www.kaggle.com/datasets/kongaevans/speaker-recognition-dataset/speaker-recognition-dataset.zip

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import shutil
import numpy as np

import tensorflow as tf
import keras

from pathlib import Path
from IPython.display import display, Audio

In [4]:
DATASET_ROOT = "16000_pcm_speeches"

# The folders in which we will put the audio samples and the noise samples
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

# Percentage of samples to use for validation
VALID_SPLIT = 0.1

# Seed to use when shuffling the dataset and the noise
SHUFFLE_SEED = 43

# The sampling rate to use.
# This is the one used in all the audio samples.
# We will resample all the noise to this sampling rate.
# This will also be the output size of the audio wave samples
# (since all samples are of 1 second long)
SAMPLING_RATE = 16000

# The factor to multiply the noise with according to:
#   noisy_sample = sample + noise * prop * scale
#      where prop = sample_amplitude / noise_amplitude
SCALE = 0.5

BATCH_SIZE = 128

## Data preparation

The dataset is composed of 7 folders, divided into 2 groups:

- Speech samples, with 5 folders for 5 different speakers. Each folder contains
1500 audio files, each 1 second long and sampled at 16000 Hz.
- Background noise samples, with 2 folders and a total of 6 files. These files
are longer than 1 second (and originally not sampled at 16000 Hz, but we will resample them to 16000 Hz).
We will use those 6 files to create 354 1-second-long noise samples to be used for training.

Let's sort these 2 categories into 2 folders:

- An `audio` folder which will contain all the per-speaker speech sample folders
- A `noise` folder which will contain all the noise samples
"""

"""
Before sorting the audio and noise categories into 2 folders,
we have the following directory structure:

```
main_directory/
...speaker_a/
...speaker_b/
...speaker_c/
...speaker_d/
...speaker_e/
...other/
..._background_noise_/
```

After sorting, we end up with the following structure:

```
main_directory/
...audio/
......speaker_a/
......speaker_b/
......speaker_c/
......speaker_d/
......speaker_e/
...noise/
......other/
......_background_noise_/
```

In [5]:
for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # If folder is `audio` or `noise`, do nothing
            continue
        elif folder in ["other", "_background_noise_"]:
            # If folder is one of the folders that contains noise samples,
            # move it to the `noise` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # Otherwise, it should be a speaker folder, then move it to
            # `audio` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

## Noise preparation

In this section:

- We load all noise samples (which should have been resampled to 16000)
- We split those noise samples to chunks of 16000 samples which
correspond to 1 second duration each

In [6]:
# Get the list of all noise files
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]
if not noise_paths:
    raise RuntimeError(f"Could not find any files at {DATASET_NOISE_PATH}")
print(
    "Found {} files belonging to {} directories".format(
        len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

Found 6 files belonging to 2 directories


Resample all noise samples to 16000 Hz

In [7]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(tf.io.read_file(path), desired_channels = 1)
    if sampling_rate == SAMPLING_RATE:
        split = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: split * SAMPLING_RATE], split)
        return sample
    else :
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None

noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), noises.shape[0], noises.shape[1] // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


In [8]:
noises[0]

<tf.Tensor: shape=(16000, 1), dtype=float32, numpy=
array([[-6.1035156e-05],
       [ 9.1552734e-05],
       [ 1.2207031e-04],
       ...,
       [ 3.0517578e-05],
       [ 3.0517578e-05],
       [ 0.0000000e+00]], dtype=float32)>

## Dataset generation

In [9]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio

def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Create a random tensor of the same size as audio ranging from
        # 0 to the number of noise stream samples that we have.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Get the amplitude proportion between the audio and the noise
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Adding the rescaled noise to audio
        audio = audio + noise * prop * scale

    return audio

def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x : path_to_audio(x), num_parallel_calls = tf.data.AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [10]:
class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class name ::", class_names)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Preprocessing Speaker ::", name)
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")]
    audio_paths += speaker_sample
    labels += [label] * len(speaker_sample)


print("Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names)))

rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

num_split_samples = int(VALID_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_split_samples))
train_audio_paths = audio_paths[: -num_split_samples]
train_labels = labels[:-num_split_samples]


print("Using {} files for training.".format(num_split_samples))
valid_audio_paths = audio_paths[-num_split_samples :]
valid_labels = labels[-num_split_samples :]

Our class name :: ['Benjamin_Netanyau', 'Nelson_Mandela', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']
Preprocessing Speaker :: Benjamin_Netanyau
Preprocessing Speaker :: Nelson_Mandela
Preprocessing Speaker :: Magaret_Tarcher
Preprocessing Speaker :: Jens_Stoltenberg
Preprocessing Speaker :: Julia_Gillard
Found 7501 files belonging to 5 classes.
Using 6751 files for training.
Using 750 files for training.


In [11]:
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size = BATCH_SIZE * 8, seed = SHUFFLE_SEED).batch(BATCH_SIZE)


valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size = 32 * 8, seed = SHUFFLE_SEED).batch(32)

In [12]:
train_ds = train_ds.map(lambda x, y : (add_noise(x, noises = noises, scale = SCALE), y), num_parallel_calls = tf.data.AUTOTUNE)
train_ds = train_ds.map(lambda x, y : (audio_to_fft(x), y), num_parallel_calls = tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size = tf.data.AUTOTUNE)

valid_ds = valid_ds.map(lambda x, y : (audio_to_fft(x), y), num_parallel_calls = tf.data.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size = tf.data.AUTOTUNE)

## Model Definition

In [13]:
def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = keras.layers.Conv1D(filters, 1, padding = "same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
    x = keras.layers.Add()([s, x])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPooling1D(pool_size = 2, strides = 2)(x)



def build_model(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape, name = "input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size = 3, strides = 3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = "relu")(x)
    x = keras.layers.Dense(128, activation = "relu")(x)
    outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "outputs")(x)
    return keras.Model(inputs = inputs, outputs = outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 8000, 1)]            0         []                            
                                                                                                  
 conv1d_1 (Conv1D)           (None, 8000, 16)             64        ['input[0][0]']               
                                                                                                  
 activation (Activation)     (None, 8000, 16)             0         ['conv1d_1[0][0]']            
                                                                                                  
 conv1d (Conv1D)             (None, 8000, 16)             32        ['input[0][0]']               
                                                                                              

In [14]:
model.compile(optimizer = "Adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "model.keras", monitor = "val_accuracy", save_best_only = True)

callbacks = [early_stopping_cb, model_checkpoint]

## Training

In [16]:
history = model.fit(
    train_ds,
    validation_data = valid_ds,
    epochs = 5,
    callbacks = callbacks)

Epoch 1/5
53/53 [==============================] - 11s 199ms/step - loss: 0.1435 - accuracy: 0.9480 - val_loss: 0.1202 - val_accuracy: 0.9587
Epoch 2/5
53/53 [==============================] - 11s 199ms/step - loss: 0.1137 - accuracy: 0.9576 - val_loss: 0.1216 - val_accuracy: 0.9640
Epoch 3/5
53/53 [==============================] - 11s 207ms/step - loss: 0.1108 - accuracy: 0.9560 - val_loss: 0.0780 - val_accuracy: 0.9747
Epoch 4/5
53/53 [==============================] - 11s 207ms/step - loss: 0.1153 - accuracy: 0.9579 - val_loss: 0.0666 - val_accuracy: 0.9787
Epoch 5/5
53/53 [==============================] - 10s 189ms/step - loss: 0.0747 - accuracy: 0.9719 - val_loss: 0.1124 - val_accuracy: 0.9587


In [17]:
print(model.evaluate(valid_ds))

24/24 [==============================] - 1s 26ms/step - loss: 0.1124 - accuracy: 0.9587
[0.11237896978855133, 0.9586666822433472]


We get ~ 98% validation accuracy.



## Demonstration

Let's take some samples and:

- Predict the speaker
- Compare the prediction with the real speaker
- Listen to the audio to see that despite the samples being noisy,
the model is still pretty accurate

In [18]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)

test_ds = test_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE)

for audios, labels in test_ds.take(1):
    # Get the signal FFT
    ffts = audio_to_fft(audios)
    # Predict
    y_pred = model.predict(ffts)
    # Take random samples
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        # For every sample, print the true and predicted label
        # as well as run the voice with the noise
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

4/4 [==============================] - 0s 23ms/step
Speaker: Julia_Gillard	Predicted: Julia_Gillard


Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau


Speaker: Magaret_Tarcher	Predicted: Benjamin_Netanyau


Speaker: Magaret_Tarcher	Predicted: Magaret_Tarcher


Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau


Speaker: Jens_Stoltenberg	Predicted: Jens_Stoltenberg


Speaker: Julia_Gillard	Predicted: Julia_Gillard


Speaker: Julia_Gillard	Predicted: Julia_Gillard


Speaker: Julia_Gillard	Predicted: Julia_Gillard


Speaker: Magaret_Tarcher	Predicted: Magaret_Tarcher
